In [ ]:
import tensorflow as tf
import string
import requests

In [ ]:
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

In [ ]:
response.text

In [ ]:
data = response.text.split('\n')
data[0]

In [ ]:
data = data[253:]

In [ ]:
data[0]

'  From fairest creatures we desire increase,'

In [ ]:
len(data)

124204

In [ ]:
data = ' '.join(data)

In [ ]:
data

In [ ]:
def clean_text(doc):
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

In [ ]:
tokens = clean_text(data)
print(tokens[:50])

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', 'feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thy']


In [ ]:
len(tokens)

898199

In [ ]:
len(set(tokens))

27956

In [ ]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i > 200000:
        break
        
print(len(lines))

199951


In [ ]:
lines[0]

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self'

Build LSTM Model and Prepare x and y

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [ ]:
sequences = np.array(sequences)
x, y = sequences[:, :-1], sequences[:,-1]

In [ ]:
x[0]

array([   47,  1408,  1264,    37,   451,  1406,     9,  2766,  1158,
        1213,   171,   132,   269,    20,    24,     1,  4782,    87,
          30,    98,  4781,    18,   715,  1263,   171,   211,    18,
         829,    20,    27,  3807,     4,   214,   121,  1212,   153,
       13004,    31,  2765,  1847,    16, 13003, 13002,   754,     7,
        3806,    99,  2430,   466,    31])

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
seq_length = x.shape[1]

LSTM Model

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            650450    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 13009)             1313909   
Total params: 2,115,259
Trainable params: 2,115,259
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x, y, batch_size = 256, epochs=60)

Epoch 1/60
782/782 [==============================] - 385s 492ms/step - loss: 6.8824 - accuracy: 0.0311
Epoch 2/60
782/782 [==============================] - 385s 493ms/step - loss: 6.5203 - accuracy: 0.0442
Epoch 3/60
782/782 [==============================] - 384s 491ms/step - loss: 6.3072 - accuracy: 0.0604
Epoch 4/60
782/782 [==============================] - 385s 492ms/step - loss: 6.1663 - accuracy: 0.0685
Epoch 5/60
782/782 [==============================] - 383s 489ms/step - loss: 6.0353 - accuracy: 0.0769
Epoch 6/60
782/782 [==============================] - 379s 484ms/step - loss: 5.9252 - accuracy: 0.0859
Epoch 7/60
782/782 [==============================] - 384s 492ms/step - loss: 5.8137 - accuracy: 0.0959
Epoch 8/60
782/782 [==============================] - 383s 490ms/step - loss: 5.7170 - accuracy: 0.1016
Epoch 9/60
782/782 [==============================] - 383s 490ms/step - loss: 5.6305 - accuracy: 0.1055
Epoch 10/60
782/782 [==============================] - 385s 492m

In [ ]:
seed_text=lines[12343]

In [ ]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []
    
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')
        
        y_predict = model.predict_classes(encoded)
        
        predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_predict:
                predicted_word = word
                break
        seed_text = seed_text + ' ' + predicted_word
        text.append(predicted_word)
    return ' '.join(text)

In [ ]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 10)

'preposterously be stained to make me to the future court'

In [ ]:
seed_text

'home of love if i have ranged like him that travels i return again just to the time not with the time exchanged so that my self bring water for my stain never believe though in my nature reigned all frailties that besiege all kinds of blood that it could so'